In [1]:
import sys
import os
import gc
import copy
import yaml
import pickle
import random
import joblib
import shutil
from time import time
import typing as tp
from pathlib import Path

import numpy as np
import pandas as pd
import scipy

from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold
from sklearn.metrics import average_precision_score as APS
import duckdb


import torch
import torchvision
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.cuda import amp
from torch.nn import BCELoss


import timm
from mamba_ssm import Mamba
from transformers import AutoModel, AutoTokenizer

import albumentations as A
from albumentations.pytorch import ToTensorV2


# use one device only
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
con = duckdb.connect()

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.11 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
class CFG:
    NUM = 20000
    TRAIN_BIND_PER = 0.1
    TEST_No = 4
    TEST_NUM = int(1674896/32 * TEST_No)
    TEST_OFFSET = int(TEST_NUM * (TEST_No-1))
    LR = 0.001
    WD = 1e-4
    NBR_FOLDS = 5
    SELECTED_FOLDS = [0, 1, 2, 3, 4]
    TRAIN_ENC_PATH = Path('/root/Kaggle_NeurIPS2024/data/external/train_enc.parquet')
    TEST_ENC_PATH = Path('/root/Kaggle_NeurIPS2024/data/external/test_enc.parquet')
    TRAIN_PATH = Path('/root/Kaggle_NeurIPS2024/data/raw/train.parquet')
    TEST_PATH = Path('/root/Kaggle_NeurIPS2024/data/raw/test.parquet')
    OUTPUT_PATH = Path(f'/root/Kaggle_NeurIPS2024/data/processed/{NUM}_50per_CLM.parquet')
    TEST_OUTPUT_PATH = Path(f'/root/Kaggle_NeurIPS2024/data/processed/test_CLM_{TEST_OFFSET}_to_{TEST_NUM}.parquet')
    BATCH_SIZE = 128
    EPOCHS = 5
    PATIENCE = 5
    REDUCE_LR_PATIENCE = 3
    REDUCE_LR_FACTOR = 0.5
    is_test = True

In [3]:
if not CFG.is_test:
    train = con.query(f"""(SELECT *
                            FROM parquet_scan('{CFG.TRAIN_PATH}')
                            WHERE binds = 0
                            ORDER BY random()
                            LIMIT {int(CFG.NUM/2)})
                            UNION ALL
                            (SELECT *
                            FROM parquet_scan('{CFG.TRAIN_PATH}')
                            WHERE binds = 1
                            ORDER BY random()
                            LIMIT {int(CFG.NUM/2)})""").df()
else:
    test = con.query(f"""(SELECT *
                        FROM parquet_scan('{CFG.TEST_PATH}')
                        LIMIT {CFG.TEST_NUM}
                        OFFSET {CFG.TEST_OFFSET}
                        )""").df()



In [4]:
if not CFG.is_test:
    display(train.head())
    display(train.tail())
else:
    display(test.head())
    display(test.tail())

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name
0,295874916,O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21,NCc1ccccc1CS(=O)(=O)N1CCOCC1,Nc1nc2c(s1)COCC2,O=C(N[Dy])C1CCC(CNc2nc(NCc3ccccc3CS(=O)(=O)N3C...,sEH
1,295874917,O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21,NCc1ccccc1N1CCOCC1,Cc1nc(CN)oc1C.Cl.Cl,Cc1nc(CNc2nc(NCc3ccccc3N3CCOCC3)nc(NCC3CCC(C(=...,BRD4
2,295874918,O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21,NCc1ccccc1N1CCOCC1,Cc1nc(CN)oc1C.Cl.Cl,Cc1nc(CNc2nc(NCc3ccccc3N3CCOCC3)nc(NCC3CCC(C(=...,HSA
3,295874919,O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21,NCc1ccccc1N1CCOCC1,Cc1nc(CN)oc1C.Cl.Cl,Cc1nc(CNc2nc(NCc3ccccc3N3CCOCC3)nc(NCC3CCC(C(=...,sEH
4,295874920,O=C(NCC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21,NCc1ccccc1N1CCOCC1,Nc1cccc2c1CN(C1CCC(=O)NC1=O)C2=O,O=C1CCC(N2Cc3c(Nc4nc(NCc5ccccc5N5CCOCC5)nc(NCC...,BRD4


,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name
209357,296084273,O=C(N[C@H](Cc1csc2ccccc12)C(=O)O)OCC1c2ccccc2-...,Cc1nc(Cl)ccc1N,CCOc1cc2c(cc1CN)OC(C)C2.Cl,CCOc1cc2c(cc1CNc1nc(Nc3ccc(Cl)nc3C)nc(N[C@H](C...,HSA
209358,296084274,O=C(N[C@H](Cc1csc2ccccc12)C(=O)O)OCC1c2ccccc2-...,Cc1nc(Cl)ccc1N,CCOc1cc2c(cc1CN)OC(C)C2.Cl,CCOc1cc2c(cc1CNc1nc(Nc3ccc(Cl)nc3C)nc(N[C@H](C...,sEH
209359,296084275,O=C(N[C@H](Cc1csc2ccccc12)C(=O)O)OCC1c2ccccc2-...,Cc1nc(Cl)ccc1N,COC(CN)C1CCOC1,COC(CNc1nc(Nc2ccc(Cl)nc2C)nc(N[C@H](Cc2csc3ccc...,BRD4
209360,296084276,O=C(N[C@H](Cc1csc2ccccc12)C(=O)O)OCC1c2ccccc2-...,Cc1nc(Cl)ccc1N,COC(CN)C1CCOC1,COC(CNc1nc(Nc2ccc(Cl)nc2C)nc(N[C@H](Cc2csc3ccc...,HSA
209361,296084277,O=C(N[C@H](Cc1csc2ccccc12)C(=O)O)OCC1c2ccccc2-...,Cc1nc(Cl)ccc1N,COC(CN)C1CCOC1,COC(CNc1nc(Nc2ccc(Cl)nc2C)nc(N[C@H](Cc2csc3ccc...,sEH


In [5]:
if not CFG.is_test:
    smiles = train['molecule_smiles']#.unique()
    print(len(smiles))
else:
    smiles = test['molecule_smiles']#.unique()
    print(len(smiles))

209362


In [6]:
# 104681 rows take about 10 minutes.
# load pre-trained ChemBERTa model checkpoint and tokenizer
cb_tokenizer = AutoTokenizer.from_pretrained('DeepChem/ChemBERTa-10M-MLM')
cb_model = AutoModel.from_pretrained('DeepChem/ChemBERTa-10M-MLM')
cb_model.eval()

# tokenize SMILES
cb_encoded_inputs = cb_tokenizer(list(smiles), padding=True, truncation=True, return_tensors="pt")

# calculate embeddings
with torch.no_grad():
    outputs = cb_model(**cb_encoded_inputs)

# extract pooled output
cb_embeddings = outputs.pooler_output

cb_embeddings_df = pd.DataFrame(cb_embeddings.numpy())
cb_embeddings_df.head()

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-10M-MLM and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# df_repeated = cb_embeddings_df.loc[cb_embeddings_df.index.repeat(3)].reset_index(drop=True)

In [ ]:
if not CFG.is_test:
    cb_embeddings_df = pd.concat([train['id'], cb_embeddings_df], axis=1)
    binds = train[['binds', 'protein_name']]
    binds['bind1'] = train.apply(lambda row: row['binds'] if row['protein_name'] == 'BRD4' else 0, axis=1)
    binds['bind2'] = train.apply(lambda row: row['binds'] if row['protein_name'] == 'HSA' else 0, axis=1)
    binds['bind3'] = train.apply(lambda row: row['binds'] if row['protein_name'] == 'sEH' else 0, axis=1)
    cb_embeddings_df = pd.concat([cb_embeddings_df, binds], axis=1)
else:
    cb_embeddings_df = pd.concat([test['id'], cb_embeddings_df], axis=1)
    cb_embeddings_df = pd.concat([cb_embeddings_df, test['protein_name']], axis=1)


In [ ]:
display(cb_embeddings_df.head())
display(cb_embeddings_df.tail())

In [ ]:
cb_embeddings_df.columns = cb_embeddings_df.columns.astype(str)

if not CFG.is_test:
    cb_embeddings_df.to_parquet(CFG.OUTPUT_PATH)
else:
    cb_embeddings_df.to_parquet(CFG.TEST_OUTPUT_PATH)